# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
# Create dataframe from data from weatherpy

cities = pd.read_csv("output_data/cities.csv", encoding="utf-8", index_col = 0)
cities.head()

,city,country,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,date
0,severomuysk,ru,56.15,113.43,64.24,55.0,46,0.13,1597117881
1,willmar,us,45.12,-95.04,64.40,72.0,1,5.55,1597117881
2,grakhovo,ru,56.05,51.97,55.40,82.0,0,8.95,1597117881
3,adet,et,11.26,37.49,55.87,86.0,100,1.86,1597117881
4,thyboron,dk,56.70,8.21,63.00,8.0,2,1.01,1597117881


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Create location and data

humidity = cities["humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["latitude", "longitude"]]

In [14]:
#create heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)

#Add heat_layer to the created figure
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# New dataframe after filtering
hotel_df = cities.loc[(cities["max_temp"] > 70) & (cities["max_temp"] < 80) & (cities["cloudiness"] == 0 & (cities['wind_speed']<10)), :]
hotel_df = hotel_df.dropna(how='any')

#Fix indices
hotel_df.reset_index(inplace=True)
del hotel_df['index']

hotel_df.head()

,city,country,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,date
0,monterey,us,36.25,-121.25,72.00,44.0,0,4.54,1597117883
1,azuaga,es,38.27,-5.68,73.99,57.0,0,3.00,1597117883
2,blackfoot,us,43.19,-112.34,73.99,16.0,0,5.01,1597117883
3,navoi,uz,40.08,65.38,78.80,29.0,0,4.70,1597117884
4,berlin,us,52.52,13.41,73.00,77.0,0,4.70,1597117884


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_list = []

#Iterate through lat/lon 
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['latitude']
    lng = hotel_df.loc[i]['longitude']

    #Establish paramaters
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    #Query URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()

    #try/catch for error handling
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        hotel_list.append("")
        
#Bring in list to hotel_df dataframe
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how='any')

#Reconfigure column names for the next cell
hotel_df.rename(columns = {'latitude':'Lat','longitude':'Lng','city':'City','country':'Country'},inplace=True)

hotel_df.head()

,City,Country,Lat,Lng,max_temp,humidity,cloudiness,wind_speed,date,Hotel Name
0,monterey,us,36.25,-121.25,72.00,44.0,0,4.54,1597117883,Bassetti Farms Shop
1,azuaga,es,38.27,-5.68,73.99,57.0,0,3.00,1597117883,Azuaga
2,blackfoot,us,43.19,-112.34,73.99,16.0,0,5.01,1597117883,Blackfoot
3,navoi,uz,40.08,65.38,78.80,29.0,0,4.70,1597117884,Navoi
4,berlin,us,52.52,13.41,73.00,77.0,0,4.70,1597117884,Berlin


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))